In [1]:
import time
import pandas as pd
import numpy as np
import math

# internal tools
from dsgtools.reporting import make_format
from dsgtools.reporting import TableWriter
from dsgtools.reporting import freq
from dsgtools.reporting import bivariate

In [25]:
path = "Analytics/RnD Projects/Product RnD/Business/LexisNexis 9999 (SBFE Inquiry POC)/Performance/Fraud Sample/fleector_11124/"

In [26]:
data = pd.read_adls(path + "fraud_model_data060822.csv", reader = pd.read_csv, dtype = str)
print(data.shape)

(444450, 92)


In [27]:
data.columns = [x.lower() for x in data.columns]

In [28]:
data.ats_id.is_unique

False

In [29]:
# data = data.drop(['unnamed: 0'], axis=1)
data = data.rename(columns = {"unnamed: 0" : "seq"})

In [30]:
print(data.shape)
data = data[data.date_created.astype(str).str.slice(0, 7).isin(["2022-01", "2022-02", "2022-03", "2022-04", "2022-05", "2022-06"])]
print(data.shape)

(444450, 92)
(83732, 92)


In [31]:
dedup_col = [x for x in data.columns if x not in ["ats_id", "seq"]]

In [32]:
data = data.fillna("")
data = data[data["app_mon"] != ""]
data = data.drop_duplicates(subset = dedup_col, ignore_index = True)
print(data.shape)
data.ats_id.is_unique

(83729, 92)


True

In [33]:
freq(data.final_fraud_flag)

,Count,Pct,Cuml Count,Cuml Pct
final_fraud_flag,,,,
0,76997.0,0.919598,76997.0,0.919598
1,6732.0,0.080402,83729.0,1.000000


In [34]:
data = data.sort_values(by = ["date_created", 'cust_name', 'cust_address1', 'cust_address2', 'cust_city', 'cust_state', 'cust_zip', 'final_fraud_flag'])
print(data.shape)
data = data.drop_duplicates(subset = ["date_created", 'cust_name', 'cust_address1', 'cust_address2', 'cust_city', 'cust_state', 'cust_zip', ], keep = "last")
print(data.shape)
freq(data.final_fraud_flag)

(83729, 92)
(83704, 92)


,Count,Pct,Cuml Count,Cuml Pct
final_fraud_flag,,,,
0,76976.0,0.919622,76976.0,0.919622
1,6728.0,0.080378,83704.0,1.000000


In [35]:
freq(data.acct_flag, data.final_fraud_flag, cross = False)

Count       Pct  Cuml Count  Cuml Pct
acct_flag final_fraud_flag                                         
0         0                 53089.0  0.634247     53089.0  0.634247
          1                  5899.0  0.070475     58988.0  0.704721
1         0                 23887.0  0.285375     82875.0  0.990096
          1                   829.0  0.009904     83704.0  1.000000

In [36]:
data.to_adls(path + "fleector_sample_to_use.parquet", overwrite=True)

In [24]:
freq(data.date_created.astype(str).str.slice(0, 7))

,Count,Pct,Cuml Count,Cuml Pct
date_created,,,,
2022-01,15827.0,0.189083,15827.0,0.189083
2022-02,15512.0,0.185320,31339.0,0.374403
2022-03,19353.0,0.231208,50692.0,0.605610
2022-04,16822.0,0.200970,67514.0,0.806580
2022-05,16185.0,0.193360,83699.0,0.999940
2022-06,5.0,0.000060,83704.0,1.000000


In [9]:
freq(data.acct_flag)

,Count,Pct,Cuml Count,Cuml Pct
acct_flag,,,,
0,281801.0,0.6342,281801.0,0.6342
1,162540.0,0.3658,444341.0,1.0000


In [14]:
# 444,341
data["count"] = 1
data = data.fillna("")
temp_roll = data.groupby(["date_created", 'cust_name', 'cust_address1', 'cust_address2', 'cust_city', 'cust_state', 'cust_zip', 'final_fraud_flag'])['count'].sum().reset_index(name='count')
print(temp_roll["count"].describe())

temp_roll["count_2"] = 1
temp_roll_2 = temp_roll.groupby(["date_created", 'cust_name', 'cust_address1', 'cust_address2', 'cust_city', 'cust_state', 'cust_zip'])['count_2'].sum().reset_index(name='count_2')
print(temp_roll_2["count_2"].describe())


count    443309.000000
mean          1.002328
std           0.050835
min           1.000000
25%           1.000000
50%           1.000000
75%           1.000000
max           5.000000
Name: count, dtype: float64
count    443262.000000
mean          1.000106
std           0.010297
min           1.000000
25%           1.000000
50%           1.000000
75%           1.000000
max           2.000000
Name: count_2, dtype: float64


In [ ]:
print(data.shape)
data = data[data.date_created.astype(str).str.slice(0, 7).isin(["2022-01", "2022-02", "2022-03", "2022-04", "2022-05", "2022-06"])]
print(data.shape)

In [11]:
freq(data.acct_flag, data.final_fraud_flag, cross = False)

Count       Pct  Cuml Count  Cuml Pct
acct_flag final_fraud_flag                                          
0         0                 258647.0  0.582091    258647.0  0.582091
          1                  23154.0  0.052109    281801.0  0.634200
1         0                 158164.0  0.355952    439965.0  0.990152
          1                   4376.0  0.009848    444341.0  1.000000

In [12]:
print(data.shape)

(444341, 91)


In [13]:
data.ats_id.is_unique

True